# Test out the pick em game before moving to its own .exe or .py

#### 2024-12-10

In [1]:
import json
import pandas as pd
import os
import random

## Load data

In [2]:
path_moves_per_pokemon = os.path.join("..", "Scraping", "data", "moves_per_pokemon_orig_names.json")
path_pokemon_main_stats = os.path.join("..", "Scraping", "data", "pokemon_main_stats_data.csv")
path_pokemon_moves_master = os.path.join("..", "Scraping", "data", "pokemon_moves_data.csv")

In [3]:
# Load moves per pokemon as dict
with open(path_moves_per_pokemon, 'r') as f:
    dict_moves_per_pokemon = json.load(f)

In [4]:
# Load dfs
## Each pokemon and their stats
df_pokemon_main_stats = pd.read_csv(path_pokemon_main_stats)

# Pokemon moves
df_pokemon_moves_master = pd.read_csv(path_pokemon_moves_master)
df_pokemon_moves_master['Accuracy'] = df_pokemon_moves_master['Accuracy'].replace('∞', '100').astype(float) # Replace '∞' with 100.0 and convert the column to floats


## Random Selection

### Randomly select two pokemon without replacement and four of their moves without replacement

In [13]:
# Randomly select two Pokémon without replacement
selected_pokemon = random.sample(list(dict_moves_per_pokemon.keys()), 2)

In [14]:
# Select 4 random moves for each Pokémon
battle_pokemon = {
    pokemon: random.sample(dict_moves_per_pokemon[pokemon], 4)
    for pokemon in selected_pokemon
}

In [15]:
battle_pokemon

{'Castform': ['Retaliate', 'Reflect Type', 'Return', 'Water Gun'],
 'Litwick': ['Shadow Ball', 'Round', 'Night Shade', 'Heat Wave']}

In [16]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == battle_pokemon['Metang'][0]]

KeyError: 'Metang'

In [52]:
df_pokemon_moves_master.Accuracy.value_counts()

100    446
90      72
∞       55
95      37
85      35
80      12
75      11
70       5
50       4
30       4
55       3
60       1
Name: Accuracy, dtype: int64

## Pokemon battle simulation

### Helper fns

In [ ]:
# # Function to calculate damage based on move and stats
# def calculate_damage(attacker_stats, defender_stats, move_stats):
#     """Calculates the damage a move inflicts based on simple rules."""
#     if move_stats['Power'] and not pd.isna(move_stats['Power']):
#         power = move_stats['Power']
#     else:
#         power = 0  # Non-damaging moves
    
#     # Damage formula (simplified for demonstration purposes)
#     attack_stat = attacker_stats['Attack'] if move_stats['Category'] == 'Physical' else attacker_stats['Sp. Atk']
#     defense_stat = defender_stats['Defense'] if move_stats['Category'] == 'Physical' else defender_stats['Sp. Def']
#     damage = (power * attack_stat / defense_stat) * random.uniform(0.85, 1.0)
#     return round(damage)

In [50]:
def calculate_damage(attacker_stats, defender_stats, move_power, effectiveness):
    if pd.isna(move_power):
        print("This move does not have defined power and deals no damage.")
        return 0  # No damage if the move has no power
    if effectiveness == 0.0:
        print("Move has no effect!")
        return 0  # No damage if the move is ineffective

    attack_stat = attacker_stats['Attack'] if attacker_stats['Attack'] > attacker_stats['Sp. Atk'] else attacker_stats['Sp. Atk']
    defense_stat = defender_stats['Defense'] if attacker_stats['Attack'] > attacker_stats['Sp. Atk'] else defender_stats['Sp. Def']

    # Pokémon damage formula
    damage = ((2 * 50 / 5 + 2) * move_power * attack_stat / defense_stat / 50 + 2) * effectiveness

    # Remove the minimum 1 damage restriction.
    return max(1, int(damage))  # Minimum of 1 damage

In [51]:
# Function to simulate a battle turn
def battle_turn(pokemon1, pokemon2, move1, move2, move_stats_df):
    # Get stats for moves
    move1_stats = move_stats_df.loc[move_stats_df["Move Name"] == move1].iloc[0]
    move2_stats = move_stats_df.loc[move_stats_df["Move Name"] == move2].iloc[0]

    # Pokémon 1 attacks
    damage_to_pokemon2 = calculate_damage(pokemon1, pokemon2, move1, move1_stats)
    pokemon2["HP"] -= damage_to_pokemon2

    # Check if Pokémon 2 fainted
    if pokemon2["HP"] <= 0:
        return f"{pokemon2['Name']} fainted! {pokemon1['Name']} wins!", None

    # Pokémon 2 attacks
    damage_to_pokemon1 = calculate_damage(pokemon2, pokemon1, move2, move2_stats)
    pokemon1["HP"] -= damage_to_pokemon1

    # Check if Pokémon 1 fainted
    if pokemon1["HP"] <= 0:
        return f"{pokemon1['Name']} fainted! {pokemon2['Name']} wins!", None

    return f"{pokemon1['Name']} used {move1}! {pokemon2['Name']} used {move2}!", (pokemon1, pokemon2)


In [52]:
# Initialize Pokémon stats
def get_pokemon_stats(name, stats_df):
    row = stats_df.loc[stats_df["Name"] == name].iloc[0]
    return {
        "Name": row["Name"],
        "Type": row["Type"],
        "HP": row["HP"],
        "Attack": row["Attack"],
        "Defense": row["Defense"],
        "Sp. Atk": row["Sp. Atk"],
        "Sp. Def": row["Sp. Def"],
        "Speed": row["Speed"],
    }

In [53]:
# # Main battle function - orig with no explicit output of each turn
# def pokemon_battle(pokemon1_name, pokemon2_name, pokemon_moves, stats_df, move_stats_df):
#     pokemon1 = get_pokemon_stats(pokemon1_name, stats_df)
#     pokemon2 = get_pokemon_stats(pokemon2_name, stats_df)

#     print(f"Battle Start: {pokemon1['Name']} vs {pokemon2['Name']}!")

#     while pokemon1["HP"] > 0 and pokemon2["HP"] > 0:
#         move1 = random.choice(pokemon_moves[pokemon1["Name"]])
#         move2 = random.choice(pokemon_moves[pokemon2["Name"]])
#         message, result = battle_turn(pokemon1, pokemon2, move1, move2, move_stats_df)
#         print(message)

#         if result is None:  # A Pokémon fainted
#             break
#         pokemon1, pokemon2 = result

In [54]:
## This does not use type effectiveness
# def pokemon_battle(pokemon1, pokemon2, pokemon_moves, df_stats, df_moves):
#     # Extract Pokémon stats
#     stats_p1 = df_stats[df_stats['Name'] == pokemon1].iloc[0]
#     stats_p2 = df_stats[df_stats['Name'] == pokemon2].iloc[0]
    
#     # Initial HP
#     hp_p1 = stats_p1['HP']
#     hp_p2 = stats_p2['HP']
    
#     # Extract moves
#     moves_p1 = pokemon_moves[pokemon1]
#     moves_p2 = pokemon_moves[pokemon2]
    
#     # Determine turn order based on Speed
#     if stats_p1['Speed'] > stats_p2['Speed']:
#         first_attacker = pokemon1
#         second_attacker = pokemon2
#     elif stats_p1['Speed'] < stats_p2['Speed']:
#         first_attacker = pokemon2
#         second_attacker = pokemon1
#     else:  # If Speed is tied, choose randomly
#         first_attacker, second_attacker = random.sample([pokemon1, pokemon2], 2)
    
#     print(f"Battle Start! {pokemon1} vs {pokemon2}\n")
#     print(f"{pokemon1} HP: {hp_p1} | {pokemon2} HP: {hp_p2}\n")
#     print(f"{first_attacker} attacks first based on Speed!\n")
    
#     # Turn-based battle
#     turn = 1
#     while hp_p1 > 0 and hp_p2 > 0:
#         print(f"--- Turn {turn} ---")
        
#         # Define attacker/defender for this turn
#         if first_attacker == pokemon1:
#             attacker, defender = pokemon1, pokemon2
#             attacker_stats, defender_stats = stats_p1, stats_p2
#             attacker_moves = moves_p1
#         else:
#             attacker, defender = pokemon2, pokemon1
#             attacker_stats, defender_stats = stats_p2, stats_p1
#             attacker_moves = moves_p2
        
#         # Attacker uses a random move
#         move = random.choice(attacker_moves)
#         move_stats = df_moves[df_moves['Move Name'] == move].iloc[0]
        
#         # Check if the move hits
#         accuracy = move_stats['Accuracy'] if not pd.isna(move_stats['Accuracy']) else 100
#         if random.uniform(0, 100) <= accuracy:
#             damage = calculate_damage(attacker_stats, defender_stats, move_stats)
#             if defender == pokemon1:
#                 hp_p1 -= damage
#                 hp_p1 = max(hp_p1, 0)
#                 print(f"{attacker} used {move} and caused {damage} damage!")
#                 print(f"{defender}'s HP is now {hp_p1}\n")
#                 if hp_p1 == 0:
#                     print(f"{defender} fainted! {attacker} wins!")
#                     break
#             else:
#                 hp_p2 -= damage
#                 hp_p2 = max(hp_p2, 0)
#                 print(f"{attacker} used {move} and caused {damage} damage!")
#                 print(f"{defender}'s HP is now {hp_p2}\n")
#                 if hp_p2 == 0:
#                     print(f"{defender} fainted! {attacker} wins!")
#                     break
#         else:
#             print(f"{attacker} used {move} but it missed!\n")
        
#         # Switch turn
#         first_attacker, second_attacker = second_attacker, first_attacker
#         turn += 1

In [55]:
def get_effectiveness(move_type, target_types, type_effectiveness):
    """
    Calculate the type effectiveness multiplier.
    move_type: Type of the move (e.g., 'Fire').
    target_types: List of the target's types (e.g., ['Grass', 'Poison']).
    type_effectiveness: Dictionary defining type interactions.
    """
    multiplier = 1.0
    for t in target_types:
        multiplier *= type_effectiveness.get(move_type, {}).get(t, 1.0)
    return multiplier

In [2]:
type_effectiveness = {
    'Normal':     {'Rock': 0.5, 'Ghost': 0.0, 'Steel': 0.5},
    'Fire':       {'Fire': 0.5, 'Water': 0.5, 'Grass': 2.0, 'Ice': 2.0, 'Bug': 2.0, 'Rock': 0.5, 'Dragon': 0.5, 'Steel': 2.0},
    'Water':      {'Fire': 2.0, 'Water': 0.5, 'Grass': 0.5, 'Ground': 2.0, 'Rock': 2.0, 'Dragon': 0.5},
    'Electric':   {'Water': 2.0, 'Electric': 0.5, 'Grass': 0.5, 'Ground': 0.0, 'Flying': 2.0, 'Dragon': 0.5},
    'Grass':      {'Fire': 0.5, 'Water': 2.0, 'Grass': 0.5, 'Poison': 0.5, 'Ground': 2.0, 'Flying': 0.5, 'Bug': 0.5, 'Rock': 2.0, 'Dragon': 0.5, 'Steel': 0.5},
    'Ice':        {'Fire': 0.5, 'Water': 0.5, 'Grass': 2.0, 'Ice': 0.5, 'Ground': 2.0, 'Flying': 2.0, 'Dragon': 2.0, 'Steel': 0.5},
    'Fighting':   {'Normal': 2.0, 'Ice': 2.0, 'Rock': 2.0, 'Dark': 2.0, 'Steel': 2.0, 'Poison': 0.5, 'Flying': 0.5, 'Psychic': 0.5, 'Bug': 0.5, 'Ghost': 0.0, 'Fairy': 0.5},
    'Poison':     {'Grass': 2.0, 'Poison': 0.5, 'Ground': 0.5, 'Rock': 0.5, 'Ghost': 0.5, 'Steel': 0.0, 'Fairy': 2.0},
    'Ground':     {'Fire': 2.0, 'Electric': 2.0, 'Grass': 0.5, 'Poison': 2.0, 'Flying': 0.0, 'Bug': 0.5, 'Rock': 2.0, 'Steel': 2.0},
    'Flying':     {'Electric': 0.5, 'Grass': 2.0, 'Fighting': 2.0, 'Bug': 2.0, 'Rock': 0.5, 'Steel': 0.5},
    'Psychic':    {'Fighting': 2.0, 'Poison': 2.0, 'Psychic': 0.5, 'Dark': 0.0, 'Steel': 0.5},
    'Bug':        {'Fire': 0.5, 'Grass': 2.0, 'Fighting': 0.5, 'Poison': 0.5, 'Flying': 0.5, 'Psychic': 2.0, 'Ghost': 0.5, 'Dark': 2.0, 'Steel': 0.5, 'Fairy': 0.5},
    'Rock':       {'Fire': 2.0, 'Ice': 2.0, 'Fighting': 0.5, 'Ground': 0.5, 'Flying': 2.0, 'Bug': 2.0, 'Steel': 0.5},
    'Ghost':      {'Normal': 0.0, 'Psychic': 2.0, 'Ghost': 2.0, 'Dark': 0.5},
    'Dragon':     {'Dragon': 2.0, 'Steel': 0.5, 'Fairy': 0.0},
    'Dark':       {'Fighting': 0.5, 'Psychic': 2.0, 'Ghost': 2.0, 'Dark': 0.5, 'Fairy': 0.5},
    'Steel':      {'Fire': 0.5, 'Water': 0.5, 'Electric': 0.5, 'Ice': 2.0, 'Rock': 2.0, 'Steel': 0.5, 'Fairy': 2.0},
    'Fairy':      {'Fire': 0.5, 'Fighting': 2.0, 'Poison': 0.5, 'Dragon': 2.0, 'Dark': 2.0, 'Steel': 0.5}
}

In [ ]:
def pokemon_battle_with_effectiveness(pokemon_1, pokemon_2, pokemon_moves, df_pokemon_main_stats, df_pokemon_moves_master, type_effectiveness):
    import random

    # Fetch Pokémon stats
    stats_1 = df_pokemon_main_stats[df_pokemon_main_stats['Name'] == pokemon_1].iloc[0]
    stats_2 = df_pokemon_main_stats[df_pokemon_main_stats['Name'] == pokemon_2].iloc[0]

    # Initialize HP
    hp_1 = stats_1['HP']
    hp_2 = stats_2['HP']

    # Start of the battle
    print(f"Battle Start: {pokemon_1} vs {pokemon_2}!")
    while hp_1 > 0 and hp_2 > 0:
        # Decide turn order
        if stats_1['Speed'] > stats_2['Speed']:
            attacker, defender = pokemon_1, pokemon_2
            attacker_moves = pokemon_moves[pokemon_1]
            defender_stats = stats_2
            defender_hp = hp_2
        else:
            attacker, defender = pokemon_2, pokemon_1
            attacker_moves = pokemon_moves[pokemon_2]
            defender_stats = stats_1
            defender_hp = hp_1

        # Select random move
        move = random.choice(attacker_moves)
        move_stats = df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == move].iloc[0]
        move_power = move_stats['Power']
        move_accuracy = move_stats['Accuracy']
        move_type = move_stats['Type']

        # Check if move has power
        if pd.isna(move_power):
            print(f"{attacker} used {move}, but it has no offensive power!")
        else:
            # Check if move hits
            if random.uniform(0, 100) <= move_accuracy:
                # Calculate effectiveness
                effectiveness = get_effectiveness(move_type, eval(defender_stats['Type']), type_effectiveness)
                damage = move_power * effectiveness

                # Apply damage
                defender_hp -= damage
                defender_hp = max(defender_hp, 0)  # Prevent negative HP

                # Print turn summary
                print(f"{attacker} used {move}! It's {effectiveness:.2f}x effective!")
                print(f"{defender} takes {damage:.2f} damage and is left with {defender_hp:.2f} HP.")
            else:
                print(f"{attacker} used {move}, but it missed!")

        # Update HP
        if attacker == pokemon_1:
            hp_2 = defender_hp
        else:
            hp_1 = defender_hp

    # Determine winner
    if hp_1 > 0:
        print(f"{pokemon_1} wins!")
    else:
        print(f"{pokemon_2} wins!")

### Battle

In [59]:
first_pokemon = next(iter(battle_pokemon))
second_pokemon = list(battle_pokemon.keys())[1]
print(first_pokemon, second_pokemon)

Castform Litwick


In [60]:
pokemon_battle_with_effectiveness(first_pokemon, second_pokemon, battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, type_effectiveness)

Battle Start: Castform vs Litwick!


ValueError: not enough values to unpack (expected 3, got 2)

In [55]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Future Sight']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
296,Future Sight,Psychic,Special,120.0,100.0,10.0,Damage occurs 2 turns later.,NaN


In [36]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Return']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
657,Return,Normal,Physical,NaN,100.0,20.0,Power increases with higher Friendship.,NaN


In [56]:
df_pokemon_main_stats[df_pokemon_main_stats['Name'] == 'Metang']

,#,Image URL,Name,Subtitle,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
460,375,https://img.pokemondb.net/sprites/scarlet-viol...,Metang,NaN,"['Steel', 'Psychic']",420,60,75,100,55,80,50


In [57]:
df_pokemon_main_stats[df_pokemon_main_stats['Name'] == 'Jigglypuff']

,#,Image URL,Name,Subtitle,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
52,39,https://img.pokemondb.net/sprites/scarlet-viol...,Jigglypuff,NaN,"['Normal', 'Fairy']",270,115,45,20,45,25,20


In [19]:
df_pokemon_main_stats[df_pokemon_main_stats['Name'] == 'Castform']

,#,Image URL,Name,Subtitle,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
429,351,https://img.pokemondb.net/sprites/scarlet-viol...,Castform,NaN,['Normal'],420,70,70,70,70,70,70
430,351,https://img.pokemondb.net/sprites/scarlet-viol...,Castform,Sunny Form,['Fire'],420,70,70,70,70,70,70
431,351,https://img.pokemondb.net/sprites/scarlet-viol...,Castform,Rainy Form,['Water'],420,70,70,70,70,70,70
432,351,https://img.pokemondb.net/sprites/scarlet-viol...,Castform,Snowy Form,['Ice'],420,70,70,70,70,70,70


In [21]:
df_pokemon_main_stats[df_pokemon_main_stats['Name'] == 'Litwick']

,#,Image URL,Name,Subtitle,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
731,607,https://img.pokemondb.net/sprites/scarlet-viol...,Litwick,NaN,"['Ghost', 'Fire']",275,50,30,55,65,55,20


In [ ]:
# Incorporate super effective, not very effective, etc.
# incorporate things like Future Sight not hitting for two turns

In [3]:
# Define the relative path to the data folder
raw_relative_path = os.path.join("..", "data", "type_effectiveness.json")

In [4]:
# Save to a file
with open(raw_relative_path, 'w') as f:
    json.dump(type_effectiveness, f, indent=4)